<h1> imports

In [42]:
file_name = "elden ring.txt"

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import tqdm as tqdm


import math
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from numpy import dot
from numpy.linalg import norm

In [43]:
from datetime import datetime
def timeNow(text):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(text, "Current Time =", current_time)

In [44]:
data = pd.read_csv("Duc_dataset_first_ref_summary.csv",index_col=0)
data

,Articles,Summaries
0,\nCambodian leader Hun Sen on Friday rejected ...,Prospects were dim for resolution of the polit...
1,\nHonduras braced for potential catastrophe Tu...,Hurricane Mitch approached Honduras on Oct. 27...
2,\nCuban President Fidel Castro said Sunday he ...,Britain caused international controversy and C...
3,"\nMUNICH, Germany (AP) _ U.S. prosecutors have...",After the bombing of U.S. embassies in East Af...
4,\nIn a critical ruling for the North American ...,In a dispute over a new collective bargaining ...
5,\nRebels attacked a village in western Uganda ...,"After years of civil war, Congo in October 199..."
6,\nIndonesian President B.J. Habibie finds atte...,Prospects for the Asia-Pacific Economic Cooper...
7,\nIsrael's Cabinet announced within hours of a...,"On Nov. 6, 1998 a suicide auto-bomb attack on ..."
8,\nBruises on the face of jailed dissident Anwa...,Malaysian Prime Minister Mahathir Mohamad rule...
9,"\nBRUSSELS, Belgium (AP) _ U.S. special envoy ...","On Oct. 4, 1998 Yugoslav President Milosevic o..."


In [45]:
from preprocessing_algorithms import*

In [46]:
sentences,filtered_sentences = process_df(data)

100%|██████████| 50/50 [01:12<00:00,  1.45s/it]


<h1> Pre Processing </h1>

In [47]:
def tokenization(text):
    sentences = sent_tokenize(text)
    filtered_sentences = remove_stop_words(sentences)
    return filtered_sentences,sentences

In [48]:
def remove_stop_words(sentences):
    # define a set of stop words 
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []
    for sentence in sentences:
        # tokenize each sentence into words
        word_tokens = word_tokenize(sentence)
        # remove all stop words from each sentence
        filtered_words = [word for word in word_tokens if not word in stop_words]
        # join all filtered words back into a single sentence
        filtered_sentence = ' '.join(filtered_words)
        filtered_sentences.append(filtered_sentence)
    return filtered_sentences

In [49]:
# f = open(file_name, "r",encoding="utf8")
# text = ""
# for line in f:
#     text += line.replace('!','.').replace('?','.').replace('\n',' ')
# f.close()

In [50]:
# filtered_sentences, sentences = tokenization(text)  # Tokenize the text into sentences and filtered sentences
# sentences

In [68]:
def summarize_with(list_of_articles,list_of_filtered_articles ,summary_algorithm):
    rows = len(list_of_articles)
    
    summarized_text = []
    for row in tqdm(range(rows)):
        sentences=list_of_articles[row]
        filtered_sentences = list_of_filtered_articles[row]
                                  #(filtered_sentences,sentence)
        summary = summary_algorithm(filtered_sentences,sentences)
        summarized_text.append(summary)
    print(summarized_text)
    summary_df = pd.DataFrame (summarized_text, columns = [f'{summary_algorithm.__name__} summary'])
    return summary_df

<h1> Text Matching 

In [52]:
def text_matching(filtered_sentences, sentences):
    # timeNow("Starting Text Matching.")
    # summary_size = 5
    # f = open(file_name, "r")
    # text = ""
    # for line in f:
    #     text += line.replace('!','.').replace('?','.').replace('\n',' ')
    # f.close()
    
    # sentences = sent_tokenize(text)  # Tokenize the text into sentences
    # sentences = remove_stop_words(sentences)
    
    word_frequencies = {}  # Create an empty dictionary to store the word frequencies
    for sentence in filtered_sentences:  # Loop through each sentence in the text
        words = nltk.word_tokenize(sentence)  # Tokenize each sentence into words
        for word in words:  # Loop through each word in the sentence
            if word not in word_frequencies.keys():  # Check if the word is already in the dictionary
                word_frequencies[word] = 1  # If not, set its count to 1
            else:
                word_frequencies[word] += 1  # If yes, increment its count by 1
    summary = []  # Create an empty list to store the summary sentences
    sentence_map = {} # Create an empty dictionary to store the sentence scores
    sent_scores = []
    sent_index = 0
    for sentence in sentences:  # Loop through each sentence in the text again
        words = nltk.word_tokenize(sentence)  # Tokenize each sentence into words again
        score = 0  # Initialize a score variable to 0
        for word in words:  # Loop through each word in the sentence
            if word in word_frequencies.keys():  # Check if the current word is present in our dictionary of word frequencies
                score += word_frequencies[word]  # If yes, add its frequency to our score variable
        sentence_map[sent_index] = [sentence]
        sent_scores.append(score)
        sent_index += 1
        
    # scores = sorted(sentence_map.keys())
    # scores = scores[len(scores)-summary_size:]
    maxScore = max(sent_scores)    
    sortedScored = list(reversed(sorted(sent_scores)))
    
    for i in range(len(sent_scores)):
        sentence_map[i].append(sent_scores[i] / maxScore)
        sentence_map[i].append(sortedScored.index(sent_scores[i]))
    
    # for score in scores:
    #     summary.append(sentence_map[score])
        # print("Score :[",score,"] Sentence :", sentence_map[score])
    # timeNow("Finished Text Matching")
    return sentence_map

In [53]:
# tm_dic = text_matching(filtered_sentences[0], sentences[0])
# tm_dic

{0: ["\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \nfor talks outside the country, accusing them of trying to ``internationalize'' \nthe political crisis.",
  0.3129556926528323,
  67],
 1: ["Government and opposition parties have asked \nKing Norodom Sihanouk to host a summit meeting after a series of post-election \nnegotiations between the two opposition groups and Hun Sen's party \nto form a new government failed.",
  0.46719012899607404,
  28],
 2: ["Opposition leaders Prince Norodom \nRanariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition \nfigures after two alleged attempts on his life, said they could not \nnegotiate freely in Cambodia and called for talks at Sihanouk's residence \nin Beijing.",
  0.25434660684240046,
  106],
 3: ['Hun Sen, however, rejected that.', 0.018227706113292205, 182],
 4: ["``I would like to make \nit clear that all meetings related to Cambodian affairs must be conducted \nin the Kingdom of Cambodia,''",
 

<h1> Luhn

In [54]:
def luhn_algorithm(filtered_sentences, sentences):
    # timeNow("Starting Luhn")
    # Initialize a list to store the summary
    summary = []
    # # Split the text into sentences
    # sentences = text.split('.')
    # Initialize a list to store the sentence scores
    sentence_scores = []
    sentence_map = {}
    sent_index = 0
    # Iterate through each sentence
    for sentence in filtered_sentences:
        # Split the sentence into words
        words = sentence.split()
        # Initialize a score for the sentence
        score = 0
        # Iterate through each word
        for word in words:
            # Calculate the score for the word
            score += len(word)
        # Add the score to the sentence scores list
        sentence_scores.append(score)
        sentence_map[sent_index] = [sentences[sent_index]]
        sent_index += 1

    maxScore = max(sentence_scores)
    sortedScored = list(reversed(sorted(sentence_scores)))
    
    for i in range(len(sentence_scores)):
        sentence_map[i].append(sentence_scores[i] / maxScore)
        sentence_map[i].append(sortedScored.index(sentence_scores[i]))
    # timeNow("Finished Luhn")
    return sentence_map

In [56]:
# luhn_dic = luhn_algorithm(filtered_sentences, sentences)

<h1> Latent Semantic Analysis

In [57]:
def create_tf_idf(filtered_sentences):
    tfidfconverter = TfidfVectorizer()
    X = tfidfconverter.fit_transform(filtered_sentences).toarray()
    return X

In [58]:
def lsa_algorithm(X):
    svdmodel = TruncatedSVD(n_components=2)
    svdmodel.fit(X)
    result = svdmodel.transform(X)
    return result

In [59]:
def lsa_summarization(filtered_sentences, sentences):
    # timeNow("Starting LSA")
    # sentences = tokenization(text)
    # filtered_sentences = remove_stop_words(sentences)
    X = create_tf_idf(filtered_sentences)
    result = lsa_algorithm(X)
    scores = result[:,1]
    summary = ""
    sentence_map = {}
    normalized = (scores-min(scores))/(max(scores)-min(scores))
    # summarize our text by selecting only those sentences with higher scores
    sortedScored = list(reversed(sorted(normalized)))
    for i in range (len (normalized)):
        sentence_map[i] = [sentences[i], normalized[i], sortedScored.index(normalized[i])]
    # timeNow("Finished LSA")
    return sentence_map

In [60]:
# lsa_dic = lsa_summarization(filtered_sentences, sentences)

In [61]:
# lsa_dic

In [62]:
#########################
#LexRank_algorithm
########

def LexRank_algorithm(filtered_sentences,sentences,size=5,threshold = 0.095):
     #creating tf_idf
    tfidfconverter = TfidfVectorizer()
    tf_idf = tfidfconverter.fit_transform(filtered_sentences).toarray()
    # y = tfidfconverter.get_feature_names_out()
    # print(y)
            
    length = len(tf_idf)
    similarity_matrix = np.zeros([length] * 2)
    
    for i in range(length):
        for j in range(i, length):
            similarity = cosine_similarity(tf_idf[i],tf_idf[j],i,j)

            if similarity:
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity    
    
    def get_summary(sentences,similarity_matrix,threshold,summary_size=1):

        if not isinstance(summary_size, int) or summary_size < 1:
            raise ValueError('\'summary_size\' should be a positive integer')

        lex_scores = rank_sentences(sentences,similarity_matrix,threshold)

        sorted_ix = np.argsort(lex_scores)[::-1]

        summary_index=[]
        for i in sorted_ix[:summary_size]:
            summary_index.append(i)
        #print(summary_index)
        return lex_scores,summary_index

    scores , summary_index = get_summary(sentences,similarity_matrix,threshold,size)

    sentence_map = {}
    normalized = (scores-min(scores))/(max(scores)-min(scores))
    # summarize our text by selecting only those sentences with higher scores
    sortedScored = list(reversed(sorted(normalized)))
    for i in range (len (normalized)):
        sentence_map[i] = [sentences[i], normalized[i], sortedScored.index(normalized[i])]
    # timeNow("Finished LSA")
    return sentence_map

    # summary = ""
    # for i in summary_index:
    #     summary += sentences[i]       
    # return summary
###############################

def connected_nodes(matrix):
    _, labels = connected_components(matrix)
    z = csr_matrix(matrix)
    groups = []
    for tag in np.unique(labels):
        #returns an array with elements from x where condition is True, and elements from y elsewhere.
        group = np.where(labels == tag)[0]
        groups.append(group)
    return groups


#cosine similarity
def cosine_similarity(list_1, list_2,i,j):
        if i == j :
            return 1
        dot = np.dot(list_1, list_2)
        if math.isclose(dot, 0):
            return 0
        norm = (np.linalg.norm(list_1) * np.linalg.norm(list_2))
        cos_sim = dot / norm
        return cos_sim
    
    
def stationary_distribution(transition_matrix,normalized=True):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)
    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix)
        distribution[group] = eigenvector
    if normalized:
        distribution /= n_1
    return distribution

# transition_matrix is a Stochastic matrix, a square matrix used to describe the transitions of a Markov chain.
def _power_method(transition_matrix):
    sentences_count = len(transition_matrix)
    eigenvector = np.ones(sentences_count)
    if len(eigenvector) == 1:
        return eigenvector
    transposed_matrix = transition_matrix.T
    lambda_val = 1.0

    while np.allclose(lambda_val, eigenvector):
        eigenvector_next = np.dot(transposed_matrix, eigenvector)
        lambda_val = np.linalg.norm(np.subtract(eigenvector_next, eigenvector))
        eigenvector = eigenvector_next
    return eigenvector


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    return weights_matrix / row_sum

def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = weights_matrix#np.zeros(weights_matrix.shape)
    #print(discrete_weights_matrix)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1
    #print(discrete_weights_matrix)

    return create_markov_matrix(discrete_weights_matrix)

def degree_centrality_scores(similarity_matrix,threshold=None,increase_power=True):
    if not (threshold is None or isinstance(threshold, float) and 0 <= threshold < 1):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None')

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(similarity_matrix,threshold)

    scores = stationary_distribution(markov_matrix,normalized=True)
    return scores

def rank_sentences(sentences,similarity_matrix,threshold=0.03):  
    scores = degree_centrality_scores(similarity_matrix,threshold)
    return scores

In [63]:
# lexRank_dic = LexRank_algorithm(filtered_sentences, sentences)

In [64]:
# lexRank_dic

In [69]:
summarize_with(sentences,filtered_sentences ,summary_algorithm=text_matching)

100%|██████████| 50/50 [00:02<00:00, 19.10it/s]

[{0: ["\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \nfor talks outside the country, accusing them of trying to ``internationalize'' \nthe political crisis.", 0.3129556926528323, 67], 1: ["Government and opposition parties have asked \nKing Norodom Sihanouk to host a summit meeting after a series of post-election \nnegotiations between the two opposition groups and Hun Sen's party \nto form a new government failed.", 0.46719012899607404, 28], 2: ["Opposition leaders Prince Norodom \nRanariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition \nfigures after two alleged attempts on his life, said they could not \nnegotiate freely in Cambodia and called for talks at Sihanouk's residence \nin Beijing.", 0.25434660684240046, 106], 3: ['Hun Sen, however, rejected that.', 0.018227706113292205, 182], 4: ["``I would like to make \nit clear that all meetings related to Cambodian affairs must be conducted \nin the Kingdom of Cambodia,''", 0.28996074032529

,text_matching summary
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


<h1> Text Rank

In [ ]:
word_embeddings = {}

In [ ]:
if not(word_embeddings):
    print("Loading Big File...")
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    print("Finished Loading Big File.")
def textRank(filtered_sentences, sentences):
    # timeNow("Starting Text Rank")
    sentence_vectors = []
    for i in filtered_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100))
        sentence_vectors.append(v)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]   
    #begining Graph stap
    import networkx as nx
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    maxScore = max(scores.values())
    for key in scores:
        scores[key] = scores[key] / maxScore
    sent_map = {}
    sortedScores = list(reversed(sorted(scores.values())))
    for index in range(len(sentences)):
        sent_map[index] = [sentences[index] , scores[index], sortedScores.index(scores[index])]
    #ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # timeNow("Finished Text Rank")
    return sent_map

Loading Big File...


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [ ]:
tr_dic = textRank(filtered_sentences, sentences)

In [ ]:
tr_dic

{0: ['Elden Ring is a 2022 action role-playing game developed by FromSoftware and published by Bandai Namco Entertainment.',
  0.8761666727633273,
  115],
 1: ['It was directed by Hidetaka Miyazaki with worldbuilding provided by the fantasy writer George R. R. Martin.',
  0.8550477757450662,
  118],
 2: ['The game was released for PlayStation 4, PlayStation 5, Windows, Xbox One, and Xbox Series X/S on February 25.',
  0.9338198722006462,
  92],
 3: ['In Elden Ring, players control a customizable player character on a journey to repair the Elden Ring and become the new Elden Lord.',
  0.975627178458103,
  15],
 4: ['The game is presented through a third-person perspective, with players freely roaming its interactive open world.',
  0.9315678393184155,
  93],
 5: ['Gameplay elements include combat using several types of weapons and magic spells, horseback riding, and crafting.',
  0.9375610443308966,
  85],
 6: ['FromSoftware wanted to create an open-world game with gameplay similar to t

In [ ]:
def createEnsembledic(tm_dic,luhn_dic,lsa_dic,tr_dic):
    ensemble_dic = {}
    finalScores = []
    for key in tm_dic:
        final_score = tm_dic[key][1] + luhn_dic[key][1]+ lsa_dic[key][1] + tr_dic[key][1]
        finalScores.append(final_score)
        scores_arr = [tm_dic[key][1],luhn_dic[key][1],lsa_dic[key][1],tr_dic[key][1]]
        sent = tm_dic[key][0]
        ensemble_dic[key] = [final_score, scores_arr, sent]
    return ensemble_dic,finalScores

In [ ]:
def ensembleSummary(ensemble_dic,finalScores,size):
    finalScores = list(reversed(sorted(finalScores)))[:]
    summaryScores = finalScores[:size] # number of summary sentences
    summary = []
    for key in ensemble_dic:
        if(ensemble_dic[key][0] in summaryScores):
            summary.append(ensemble_dic[key][2])
    return summary

<h1> Summarizing the dataset

In [ ]:
def extractSummaryFromDic(dic, size):
    summaryScores = []
    summarySent = []
    for i in range(size):
        summaryScores.append(dic[i][1])
        summarySent.append(dic[i][0])
        
    for i in range(size, len(dic), 1):
        if(dic[i][1] > min(summaryScores)):
            ind = summaryScores.index(min(summaryScores))
            summaryScores[ind] = dic[i][1]
            summarySent[ind] = dic[i][0]
    return summarySent

In [ ]:
def getTextRanks(filtered_sentences, sentences):
    # timeNow("Started Document")
    # timeNow("Text Matching")
    tm_dic = text_matching(filtered_sentences, sentences)
    # timeNow("Luhn")
    luhn_dic = luhn_algorithm(filtered_sentences, sentences)
    # timeNow("LSA")
    lsa_dic = lsa_summarization(filtered_sentences, sentences)
    # timeNow("Text Rank")
    tr_dic = textRank(filtered_sentences, sentences)
    # timeNow("Ensemble")
    ensemble_dic, ensemble_scores = createEnsembledic(tm_dic, luhn_dic, lsa_dic, tr_dic)
    # timeNow("Finished Document")
    return ensemble_dic, ensemble_scores, tm_dic, luhn_dic, lsa_dic, tr_dic

In [ ]:
data = pd.read_csv('news_summary.csv', encoding= 'ISO-8859-1')
data.head(1)

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...


In [ ]:
data.drop(["author","date","headlines","read_more"], axis = 1, inplace = True)
data.head(1)

,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...


In [ ]:
# data = pd.read_csv('news_summary_more.csv', encoding= 'ISO-8859-1')
# data.head(1)

In [ ]:
# data.drop(["headlines"], axis = 1, inplace = True)
# data.head(1)

In [ ]:
data.sample().values[0][0]

'Actor Sushant Singh Rajput has said that if he\'s attending an award show and doesn\'t win an award, he feels slightly awkward. "A bit of acknowledgment from you folks could be a nice feeling," he added. Notably, Sushant was named Critics\' Choice Best Actor at Star Screen Awards for playing the titular role in \'M.S. Dhoni: The Untold Story\'. '

In [ ]:
negWeights = []
x = 0
for row in data.iterrows():
    text = row[1]["text"]
    sumSize = len(sent_tokenize(row[1]["text"]))
    timeNow("Document "+str(x))
    x += 1
    try:
        filtered_sentences, sentences = tokenization(text)
        ensemble_dic, ensemble_scores, tm_dic, luhn_dic, lsa_dic, tr_dic = getTextRanks(filtered_sentences, sentences)
        goldenSents = ensembleSummary(ensemble_dic, ensemble_scores, sumSize)
    except:
        continue
    
    goldenSentsInd = []
    
    w = [0,0,0,0] #tm, luhn, lsa, tr
    # print(goldenSents)
    for i in range(len(sentences)):
        if(sentences[i] in goldenSents):
            goldenSentsInd.append(i)
    # print(goldenSentsInd)
    
    for i in range(len(goldenSentsInd)):
        gInd = goldenSentsInd[i]
        goldenSents[i] = [goldenSents[i], tm_dic[gInd][2], luhn_dic[gInd][2], lsa_dic[gInd][2], tr_dic[gInd][2]]
        
        w[0] += i - tm_dic[gInd][2]
        w[1] += i - luhn_dic[gInd][2]
        w[2] += i - lsa_dic[gInd][2]
        w[3] += i - tr_dic[gInd][2]
        
    negWeights.append(w)

Document 0 Current Time = 20:15:00
Document 1 Current Time = 20:15:00
Document 2 Current Time = 20:15:00
Document 3 Current Time = 20:15:00
Document 4 Current Time = 20:15:00
Document 5 Current Time = 20:15:00
Document 6 Current Time = 20:15:00
Document 7 Current Time = 20:15:00
Document 8 Current Time = 20:15:00
Document 9 Current Time = 20:15:00
Document 10 Current Time = 20:15:00
Document 11 Current Time = 20:15:00
Document 12 Current Time = 20:15:00
Document 13 Current Time = 20:15:00
Document 14 Current Time = 20:15:00
Document 15 Current Time = 20:15:00
Document 16 Current Time = 20:15:00
Document 17 Current Time = 20:15:00
Document 18 Current Time = 20:15:00
Document 19 Current Time = 20:15:00
Document 20 Current Time = 20:15:00
Document 21 Current Time = 20:15:00
Document 22 Current Time = 20:15:00
Document 23 Current Time = 20:15:00
Document 24 Current Time = 20:15:01
Document 25 Current Time = 20:15:01
Document 26 Current Time = 20:15:01
Document 27 Current Time = 20:15:01
Do

F:\Program Files\Anaconda\lib\site-packages\sklearn\decomposition\_truncated_svd.py:268: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


Document 1424 Current Time = 20:15:23
Document 1425 Current Time = 20:15:23
Document 1426 Current Time = 20:15:23
Document 1427 Current Time = 20:15:23
Document 1428 Current Time = 20:15:23
Document 1429 Current Time = 20:15:23
Document 1430 Current Time = 20:15:23
Document 1431 Current Time = 20:15:24
Document 1432 Current Time = 20:15:24
Document 1433 Current Time = 20:15:24
Document 1434 Current Time = 20:15:24
Document 1435 Current Time = 20:15:24
Document 1436 Current Time = 20:15:24
Document 1437 Current Time = 20:15:24
Document 1438 Current Time = 20:15:24
Document 1439 Current Time = 20:15:24
Document 1440 Current Time = 20:15:24
Document 1441 Current Time = 20:15:24
Document 1442 Current Time = 20:15:24
Document 1443 Current Time = 20:15:24
Document 1444 Current Time = 20:15:24
Document 1445 Current Time = 20:15:24
Document 1446 Current Time = 20:15:24
Document 1447 Current Time = 20:15:24
Document 1448 Current Time = 20:15:24
Document 1449 Current Time = 20:15:24
Document 145

In [ ]:
    ensemble_sum = ensembleSummary(ensemble_dic, ensemble_scores, sumSize)
    tm_sum = extractSummaryFromDic(tm_dic, sumSize)
    luhn_sum = extractSummaryFromDic(luhn_dic, sumSize)
    lsa_sum = extractSummaryFromDic(lsa_dic, sumSize)

In [ ]:
len(negWeights)

4513

In [ ]:
df = pd.DataFrame(negWeights, columns=['tm','luhn','lsa', 'tr'])
df.to_csv('weights.csv', index=False)

In [ ]:
negWeightsSum = list(map(abs,[sum(x) for x in zip(*negWeights)]))

In [ ]:
negWeightsSum

[463, 109, 5, 1055]

In [ ]:
negWeightAvg = [0,0,0,0]
for i in range(len(negWeightsSum)):
    negWeightAvg[i] = round(negWeightsSum[i]/len(negWeights), 2)

In [ ]:
negWeightAvg

[0.1, 0.02, 0.0, 0.23]